# SARA MALALAN

## Homework 3

In [7]:
import numpy as np
from utils.misc import get_params_num, get_accuracy

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from IPython import display

torch.manual_seed(2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

Device: cpu


In [8]:
# import CIFAR10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

print("CIFAR images shape: {}".format(tuple(trainset[0][0].shape)))

Files already downloaded and verified
Files already downloaded and verified
CIFAR images shape: (3, 32, 32)


In [36]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.input_dim = 3 * 32 * 32
        self.n_classes = 10
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 128, kernel_size=3)
        self.avgpool = nn.AvgPool2d(kernel_size=2)
        self.out = nn.Linear(128 * 6 * 6, self.n_classes)
        
    def forward(self, x, verbose=False):
        x = F.selu(self.conv1(x))
        x = self.avgpool(x) 
        x = F.celu(self.conv2(x))
        x = self.avgpool(x) 
        x = x.view(-1, 128 * 6 * 6)
        x = self.out(x)
        return x
        
        
net = CNN()
net.to(device)
print("# of parameters: {}".format(get_params_num(net)))
print(net)

# of parameters: 83978
CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (out): Linear(in_features=4608, out_features=10, bias=True)
)


In [37]:
lr = 0.001
#momentum = 0.9
epochs = 20

n_batches = len(trainloader)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)


net.train() 
for e in range(epochs):
    for i, data in enumerate(trainloader):
        batch = data[0].to(device)
        labels = data[1].to(device)      
        outputs = net(batch)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 50 == 0:
            print("[EPOCH]: {}, [BATCH]: {}/{}, [LOSS]: {}".format(e, i, n_batches, loss.item()))
            display.clear_output(wait=True)

[EPOCH]: 19, [BATCH]: 750/782, [LOSS]: 0.963545560836792


In [38]:
acc_train = get_accuracy(trainloader, net, device=device)
acc_test = get_accuracy(testloader, net, device=device)
print("Train accuracy: {}\nTest accuracy: {}".format(acc_train, acc_test))

Train accuracy: 0.70642
Test accuracy: 0.6418
